# Stereovision

![Suzanne](main.png)

Stereovision is a discipline that deals with the reconstruction of 3D information from images. For the reconstruction of a point, several images of this point are needed. These images must be taken from different points of view. The key step of the reconstruction, which is often problematic, is to identify the images of the point to be reconstructed in each view.

## Epipolar Geometry

Epipolar geometry involves two cameras. The epipolar geometry describes the geometric properties between two views of the same scene and depends only on the intrinsic parameters of the cameras and their relative positions. It provides, in particular, the epipolar constraint, which will be very useful to produce the matches between views.

## The Fondamental Matrix

![Epipolar Geometry - Sanyam Kapoor](epipolar.png)

Let us imagine that we have two images, right and left, of the world space. Let's take a point $\vec{x}$ in the right image space. The point $\vec{X}$ of the world space, of which $\vec{x}$ is the image, can be anywhere on the line passing through $\vec{x}$ and the optical center of the right camera. We will call this line the back-projected ray of $\vec{x}$. Let us note $\vec{x}'$ the image of $\vec{X}$ in the left image space. The locus of $\vec{x}'$ is therefore the image line of the back-projected ray of $\vec{x}$. This line is called the epipolar line and is denoted $\vec{l}'$. The epipolar line passes through the epipole $\vec{e}'$, image of the optical center of the right camera.

In 2D projective geometry, a line with equation $ax+by+c = 0$ is represented by a vector with three components $(a, b, c)^T$ defined to within one factor. Thus, we have the following relationship:

>The point $\vec{x}$ belongs to the line $\vec{l}$ if and only if $x^T\vec{l} = 0$.

Moreover, in 2D projective geometry, the following remarkable relations are valid:

- The intersection of two lines $l$ and $l'$ is given by $x = l \times l'$,
- The line passing through two points $x$ and $x'$ is given by $l = x \times x'$.

Note that the vector product can be written as a product of matrix $x \times y = [x]_\times y$ where

$$[x]_\times = \begin{pmatrix} 0 & −x3 & x2 \\ x3 & 0 & −x1 \\ −x2 & x1 & 0 \end{pmatrix}$$

To find the equation of the epipolar line in the left image space, we just need to find the coordinates of two points of this line. The first is the image $P'\vec{C}$ of the optical center $\vec{C}$ of the right camera where $P'$ is the projection matrix of the left camera. The second is $P'P^{+}\vec{x}$ where $P^{+}$ is the pseudo inverse of the projection matrix $P$ of the right camera. The epipolar line thus has the equation $l' = [P'\vec{C}]_\times{}P'P^{+}\vec{x} = F\vec{x}$ with $F = [P'\vec{C}]_\times{}P'P^{+}$. $F$ is called fundamental matrix.

Since the epipolar line $\vec{l}' = F\vec{x}$ is the locus of $\vec{x}'$, $\vec{x}'$ therefore belongs to $\vec{l}'$ which leads to the epipolar constraint :

>**The fundamental matrix is such that for any pair of points corresponding $\vec{x} \leftrightarrow \vec{x}'$ in the two images, we have $\vec{x}'^{T}F\vec{x} = 0$.**

## Computation of the fundamental matrix

The fundamental matrix $F$ has seven degrees of freedom. It has nine components but these are defined to within one scale factor, which removes one degree of freedom. Moreover, the matrix $F$ is a singular matrix ($det(F) = 0$) which gives us seven degrees of freedom. So we need at least seven correspondences to compute $F$. The equation $x'^{T}_iFx_i = 0$ and the seven correspondences allow us to write a system of equations of the form $Af = 0$, where $f$ is the vector which contains the components of the matrix $F$. Let us assume that $A$ is a 7×9 matrix of rank 7. The general solution of $Af = 0$ can be written $\alpha f_1 + (1-\alpha) f_2$ where $f_1$ and $f_2$ are two particular independent solutions of $Af = 0$. We then use the singularity constraint $det(\alpha F_1 + (1 - \alpha)F_2) = 0$ to determine $\alpha$. Since the singularity constraint gives rise to a third degree equation, we may have one or three solutions for $F$.

## OpenCV

In practice you will use the OpenCV library. In python, you have access to its functions through the `cv2` module.

You can find help with the calibration and reconstruction functions on the site https://docs.opencv.org/4.0.0/d9/d0c/group__calib3d.html

## Goal

In the zip of the statement you will find two sequences of images taken by two cameras during the scanning of an object by a laser plane.

![Laser](scanRight/scan0010.png)

You will also find shots of a checkerboard in different positions that will help you calibrate your cameras.

![Damier](chessboards/c2Right.png)

The goal is to reconstruct the scanned object in 3D.

In [2]:
"""
1 : calibration de la camera
2 : par cette calibration, on peut récupérer les matrices de rotations et de translations correspondantes pour chaque image 


1 : trouver les matrices projections correspondantes des deux camera P et P' (par calibration de caméra) 
On a mis une ligne rouge sur l'objet avant de prendre des paires de photos (gauche et droite)
2 : on peut determiner la matrice fondamentale a partir à partir de P et P', la matrice fondamentale F permet de trouver la ligne épipolaire (sur l'image de droite) correspondante à un point de l'image de gauche 

Avant de prendre les deux photos des cameras, on trace une ligne rouge sur l'objet qui va permettre de faciliter la correspondances entre un point et sa ligne épipolaire
draw_epilines() dans open cv
"""


import numpy as np
from numpy.linalg import inv
import cv2 as cv
import glob

def calibration():
    # termination criteria
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((7*7,3), np.float32)
    objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.
    images = glob.glob('./chessboards/*.png')
    for fname in images:
        img = cv.imread(fname)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        # Find the chess board corners
        ret, corners = cv.findChessboardCorners(gray, (7,7), None)
        # If found, add object points, image points (after refining them)
        if ret == True:
            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray,corners, (7,7), (-1,-1), criteria)
            imgpoints.append(corners2)
            # Draw and display the corners
            cv.drawChessboardCorners(img, (7,7), corners2, ret)
            # cv.imshow('img', img)
            # cv.waitKey(500)
    cv.destroyAllWindows()
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return ret,mtx,dist,rvecs,tvecs

    


In [3]:
"""
mtx est la matrice intrinsèque de la camera -> Matrice 3x3
rvecs : est un tuple d'array qui contient les composantes rotationnelles
tvecs : idem mais pour translation
"""

ret,mtx,dist,rvecs,tvecs = calibration()


def get_projection_matrix(i):
    R = cv.Rodrigues(rvecs[i])[0]
    t = tvecs[i]
    Rt = np.concatenate([R,t], axis=-1) # [R|t] -> matrice de rotation
    P = np.matmul(mtx,Rt) # A[R|t]
    return P

def get_camera_centre_matrix(i):
    R = cv.Rodrigues(rvecs[i])[0]
    t = tvecs[i]
    #C = -R(eventuellement transposée) @ t
    Rt = np.concatenate([R,t], axis=-1) # [R|t] -> matrice de rotation
    C = -R@t # est une 3x1, on ajoute une quatrième composante qu'on fixe à 1 pour une caméra centrée dans le repère world
    C = np.vstack([C,np.array([1])])
    return C
"""
Create a dictionnary containning all interesting matrixes
"""
matrixes = dict()
matrixes['mtx'] = mtx
matrixes['rvecs'] = rvecs
matrixes['tvecs'] = tvecs

matrixes['P'],matrixes['C']= get_projection_matrix(0),get_camera_centre_matrix(0)
matrixes["P'"],matrixes["C'"]= get_projection_matrix(1),get_camera_centre_matrix(1)
matrixes["PseudoP"] = np.linalg.pinv(matrixes['P'])
# here we are only considerating the first set of 2 pictures


def crossMat(vec):# doit retourner
    x,y,z = vec
    M = np.array([[0,-z[0],y[0]],
                  [z[0],0,-x[0]],
                  [-y[0],x[0],0]])
    return M

"""
Compute first factor for Fundamental matrix
"""
First = matrixes["P'"] @ matrixes['C']
First = crossMat(First)
"""
Compute second factor for Fundamental matrix
"""
Second = matrixes["P'"]@ matrixes['PseudoP']
"""
Matrice Fondamentale
"""
print("Fundamental")
F = First @Second
print(F)


# F = [P'C]x  P'  P+
#     3x4 4x1 3x4 4x3
 
#on a que l = F . x (l = ligne épipolaire, et x coordonnée du point image issue d'une image Left ou Right)


    




Fundamental
[[ 7.46671675e+00 -3.11373946e+01 -2.64931378e+04]
 [ 3.53553100e+01  2.34629570e+00 -5.95894922e+04]
 [ 8.72536137e+03  5.53836121e+04  9.39038580e+06]]


In [4]:
import cv2
import numpy as np

def display_lines(img,lines):
    for line in lines:
        x1,y1,x2,y2 = line
        cv2.line(img,(x1,y1),(x2,y2),(0,255,0))
        break



# Load the image
img = cv2.imread('./scanLeft/0003.png')
img = cv2.imread('./scanRight/scan0003.png')

# Convert the image to the HSV color space
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Set the lower and upper bounds for the red values
lower_red = (0, 50, 50)
upper_red = (10, 255, 255)

# Create the mask using the cv2.inRange() function
gray = cv2.inRange(hsv, lower_red, upper_red)

lines = cv2.HoughLinesP(gray,1,np.pi/180,50,minLineLength=80,maxLineGap=300)
# print(lines.shape) (6,1,4) on a dectecté 6 lignes , le 4 correspond aux quatres coordonnées x1,y1,x2,y2 
lines = np.squeeze(lines)# pour retirer la dimension inutile


display_lines(img,lines)

print(lines[0])
x1,y1,x2,y2 = lines[0]



# Display the original and filtered images
cv2.imshow('Original', img)
# cv2.imshow('Filtered', gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

[1497  440 1503  551]
